In [1]:
import pandas as pd
import os
import numpy as np
import re

In [4]:
# Arbeitsverzeichnis ändern 
#neues_verzeichnis = "c:\\Users\\lpres\\OneDrive - JGU\\WMDV\\WebMining"
neues_verzeichnis = "C:\\Users\\Lucian\\Desktop\\WebMining"
os.chdir(neues_verzeichnis)

# Überprüfen, ob das Verzeichnis geändert wurde
print("Neues Arbeitsverzeichnis:", os.getcwd())

Neues Arbeitsverzeichnis: C:\Users\Lucian\Desktop\WebMining


In [11]:
file = "owned_games_5.csv"

df = pd.read_csv(file, dtype={
         "steam_64_id": str,
         "appid": "Int64",  
         "playtime_forever": "Int64",
         "playtime_2weeks": "Int64"
    })

print(df.shape)

unique_df = df.drop_duplicates()

print(unique_df.shape)

(939957, 6)
(937275, 6)


In [3]:
# Alle vorhandenen owned_games_x Dateien finden
owned_files = [f for f in os.listdir('.') if re.match(r'owned_games_\d+\.csv', f)]
owned_files.sort(key=lambda x: int(re.search(r'\d+', x).group()))

print(f"Vorhandene Dateien: {owned_files}")

if not owned_files:
    print("Keine Dateien im Format 'owned_games_x.csv' gefunden. Das Skript wird beendet.")

# Alle Dateien einlesen und in einem DataFrame speichern
df = pd.DataFrame()  # Leerer DataFrame

for file in owned_files:
    file_df = pd.read_csv(file, dtype={
         "steam_64_id": str,
         "appid": "Int64",  
         "playtime_forever": "Int64",
         "playtime_2weeks": "Int64"
    })
    df = pd.concat([df, file_df], ignore_index=True)

print("alle Dateien eingelesen")

Vorhandene Dateien: ['owned_games_1.csv', 'owned_games_2.csv', 'owned_games_3.csv', 'owned_games_4.csv', 'owned_games_5.csv']


C:\Users\lpres\AppData\Local\Temp\ipykernel_12552\1367417926.py:14: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  file_df = pd.read_csv(file, dtype={


alle Dateien eingelesen


In [16]:
# Ausgabe der ersten Zeilen
print(df.head())
print(df.shape)

         steam_64_id   appid                    name  playtime_forever  \
0  76561198217215311  109410           Brawl Busters                 0   
1  76561198217215311  304930                Unturned                 0   
2  76561198217215311  339280                  Strife                 0   
3  76561198217215311  444090                Paladins                 0   
4  76561198217215311  596350  Paladins - Public Test                 0   

   playtime_2weeks has_community_visible_stats  
0                0                        True  
1                0                        True  
2                0                        True  
3                0                        True  
4                0                       False  
(5024846, 6)


In [17]:
# Größe des Datensatzes
dataset_size = df.shape

# Anzahl der Spieler
player_count = df['steam_64_id'].nunique()

# Anzahl der einzigartigen Spiele (basierend auf AppID)
unique_games_count = df['appid'].nunique()

# Spiele, die von den meisten Spielern gespielt werden
most_played_games = df['name'].value_counts().head(10)

# Spiele mit der meisten Spielzeit (gemessen an "playtime_forever")
most_played_time_games = df.groupby('name')['playtime_forever'].sum().sort_values(ascending=False).head(10)
# groupby --> ein Eintrag pro Spiel; playtime_forever summiert für alle Spieler

most_playing_player = df.groupby('steam_64_id')['playtime_forever'].sum().sort_values(ascending=False).head(10)


print('Größe des Datensatzes: ', dataset_size)
print('Anzahl der Spieler: ', player_count)
print('Anzahl der einzigartigen Spiele: ', unique_games_count)
print('Spiele, die von den meisten Spielern gespielt werden: ', most_played_games)
print('Spiele mit der meisten Spielzeit: ', most_played_time_games)
print('Spieler mit der meisten Spielzeit: ', most_playing_player)


Größe des Datensatzes:  (5024846, 6)
Anzahl der Spieler:  35021
Anzahl der einzigartigen Spiele:  36617
Spiele, die von den meisten Spielern gespielt werden:  name
Counter-Strike 2         35598
Dota 2                   33951
PUBG: BATTLEGROUNDS      27659
Wallpaper Engine         17341
Left 4 Dead 2            17266
Apex Legends             17146
PAYDAY 2                 13734
Warframe                 13600
Dota Underlords          13354
Don't Starve Together    12826
Name: count, dtype: int64
Spiele mit der meisten Spielzeit:  name
Dota 2                            6930350825
Counter-Strike 2                  1616206994
PUBG: BATTLEGROUNDS                450094073
Grand Theft Auto V                 219283819
Rust                               184708641
Apex Legends                       137958225
Tom Clancy's Rainbow Six Siege     118049317
Warframe                           100389571
Wallpaper Engine                    79939484
Team Fortress 2                     78496349
Name: play

Daten für Vergleichsdashboard

In [10]:
#Daten für Vergleich filtern

#Rausfiltern aller Spieler, die eins der folgenden Spiele mindestens 10 stunden gespielt haben und has_community_visible_stats = True
#appid": 730, "name": "Counter-Strike 2"
#appid": 240, "name": "Counter-Strike: Source"
#appid": 648800, "name": "Raft"
#appid": 252490, "name": "Rust"
#appid": 221100, "name": "DayZ"
#appid": 218620, "name": "Payday2"
#appid": 222880, "name": "Insurgency"

games = [730, 240, 648800, 252490, 221100, 218620, 222880]

df_filtered = df[(df['appid'].isin(games)) & (df['playtime_forever'] >= 600) & (df['has_community_visible_stats'] == True)]




In [18]:
# Größe des Datensatzes
dataset_size = df_filtered.shape

# Anzahl der Spieler
player_count = df_filtered['steam_64_id'].nunique()

# Anzahl der einzigartigen Spiele (basierend auf AppID)
unique_games_count = df_filtered['appid'].nunique()

# Spiele, die von den meisten Spielern gespielt werden
most_played_games = df_filtered['name'].value_counts().head(10)

# Spiele mit der meisten Spielzeit (gemessen an "playtime_forever")
most_played_time_games = df_filtered.groupby('name')['playtime_forever'].sum().sort_values(ascending=False).head(10)
# groupby --> ein Eintrag pro Spiel; playtime_forever summiert für alle Spieler

print('Größe des Datensatzes: ', dataset_size)
print('Anzahl der Spieler: ', player_count)
print('Anzahl der einzigartigen Spiele: ', unique_games_count)
print()
print('Spiele, die von den meisten Spielern gespielt werden: ', most_played_games)
print('Spiele mit der meisten Spielzeit: ', most_played_time_games)

Größe des Datensatzes:  (43352, 6)
Anzahl der Spieler:  19778
Anzahl der einzigartigen Spiele:  7

Spiele, die von den meisten Spielern gespielt werden:  name
Counter-Strike 2          26102
PAYDAY 2                   5122
Rust                       4802
Raft                       2579
DayZ                       1933
Counter-Strike: Source     1774
Insurgency                 1040
Name: count, dtype: int64
Spiele mit der meisten Spielzeit:  name
Counter-Strike 2          1615245642
Rust                       184306236
PAYDAY 2                    48809595
Counter-Strike: Source      48665205
DayZ                        46192506
Raft                         7615038
Insurgency                   4576079
Name: playtime_forever, dtype: Int64


In [12]:
#Daten für Spielervergleich speichern
df_filtered.to_csv('filtered_data.csv', index=False)